# Eflux2 applied to *A. niger*

In [3]:
import sys
sys.path.append('../src')
from eflux2 import EFlux2
import cobra
import pandas as pd

## Read  global proteomics normalized to strain 7 

[panD+,bapat+, hpdh+,pyc+,$\Delta$ald6 (3HP+PYC+3op1D)](https://registry.agilebiofoundry.org/entry/8348)**

In [5]:
global_prot = pd.read_csv('../data/normalized_to_3HP+PYC+3op1D_global_jgi.csv',index_col='Transcript')
global_prot

,SF ABF93_1-R1,SF ABF93_1-R2,SF ABF93_1-R3,SF ABF93_10-R1,SF ABF93_10-R2,SF ABF93_10-R3,SF ABF93_11-R1,SF ABF93_11-R2,SF ABF93_11-R3,SF ABF93_12-R1,...,SF ABF93_7-R1,SF ABF93_7-R2,SF ABF93_7-R3,SF ABF93_8-R1,SF ABF93_8-R2,SF ABF93_8-R3,SF ABF93_9-R1,SF ABF93_9-R2,SF ABF93_9-R3,ID
Transcript,,,,,,,,,,,,,,,,,,,,,
PAND_Tribolium_castaneum,0.897869,0.908833,0.889132,1.018605,1.014299,1.017768,0.996457,0.999091,0.997239,1.020995,...,1.005098,1.005080,1.0,0.997879,1.000489,0.994095,1.004779,1.002646,1.002474,PAND_Tribolium_castaneum
BAPAT_Bacillus_cereus,0.873266,0.881362,0.866362,1.001068,1.000740,1.000919,1.005619,1.004879,1.002993,0.999591,...,1.002091,1.003018,1.0,1.000204,0.997112,1.001840,1.006528,1.002281,1.005291,BAPAT_Bacillus_cereus
HPDH_escherichia_coli,0.896059,0.900158,0.876699,1.017093,1.012390,1.014003,0.996349,0.998766,0.995981,1.016578,...,1.006182,1.009498,1.0,1.000692,0.999978,0.996536,1.004999,0.998779,1.001109,HPDH_escherichia_coli
1161525,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161249
1141544,1.025211,1.017555,1.019738,1.005094,1.003461,1.002065,1.004549,1.008831,1.011385,1.003238,...,1.001992,0.997855,1.0,0.998752,1.002399,1.001372,1.003830,1.003504,1.004176,1141268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097858,1.010888,1.042946,0.995899,0.992048,0.981758,0.955985,0.998675,0.963280,0.999036,1.025474,...,0.986650,0.969747,1.0,0.987936,0.993080,0.987141,0.992253,1.083444,1.132704,1097582
1161049,1.006085,1.011854,1.002625,1.005603,1.000959,1.004930,1.005106,1.005667,1.002879,1.002241,...,1.000432,1.001195,1.0,0.999567,1.002763,0.999103,1.007591,1.000758,0.994921,1160773
119417,0.977984,1.016760,1.000364,0.982975,1.001572,0.990764,0.951591,1.005419,1.001910,0.996526,...,1.000628,0.951815,1.0,0.964085,0.981043,1.004765,0.972454,1.000000,0.978223,119417


## Read iJB1325 with non-native genes added for 3HP production

In [7]:
model = cobra.io.load_json_model('../models/iJB1325_HP.nonnative_genes.pubchem.flipped.nonzero.reduced.json')
model.solver = 'gurobi'
model.solver

Using license file /Users/zuck016/gurobi.lic
Set parameter TokenServer to value leghorn.emsl.pnl.gov


In [8]:
model.summary()

Non-linear or non-reaction model objective. Falling back to minimal display.


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [9]:
model.reactions.BOUNDARY_CO2e.compartments

{'e'}

In [10]:
model.objective='BOUNDARY_He'
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
GLCe,BOUNDARY_GLCe,1000,6,100.00%
O2e,BOUNDARY_O2e,1000,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ACe,BOUNDARY_ACe,-1000,2,33.33%
He,BOUNDARY_He,-1000,0,0.00%
TARe,BOUNDARY_TARe,-1000,4,66.67%


## Map experiment IDs to Line IDs

In [11]:
experiment = pd.read_csv('experiment_w_reps.tsv', sep='\t')
experiment['ICE']= experiment['Strain'].str.replace('https://registry.agilebiofoundry.org/entry/', 'ABF_00')
experiment = experiment.sort_values(['ICE','Line Name']).dropna().set_index('Line Name')
experiment

FileNotFoundError: [Errno 2] No such file or directory: 'experiment_w_reps.tsv'

In [7]:
line_rates = pd.read_csv('a.niger_strain_specific_rates.csv')
line_rates = line_rates.sort_values(['Strain (ICE)', 'Omics Sample ID']).dropna()
line_rates.index = experiment.index
line_rates.to_csv('Line_rates.csv')

In [8]:
strain_rates = line_rates.groupby(['Strain (ICE)', 'Genotype']).mean()
strain_rates_std = line_rates.groupby(['Strain (ICE)', 'Genotype']).std()
strain_rates.to_csv('Strain_rates_mean.csv')
strain_rates_std.to_csv('Strain_rates_std.csv')

In [6]:
ref_line = 'SF ABF93_7-R3'
ref_strain = 'ABF_008348'
ref_line_glucose_rate = line_rates.loc[ref_line,'glucose_uptake_rates (mmol/gDCW * hr)']
ref_line_glucose_rate

19.890188397376424

## Normalize uptake and secretion rates by glucose uptake rate of reference study line.


In [7]:
normalized_line_rates = line_rates.drop(['Omics Sample ID', 'Strain (ICE)', 'Genotype'], axis=1)
normalized_line_rates = normalized_line_rates.divide(normalized_line_rates.loc[ref_line, 'glucose_uptake_rates (mmol/gDCW * hr)'], axis=1)
normalized_line_rates.to_csv('Learn/normalized_line_rates.csv')
normalized_line_rates

,growth_rate,glucose_uptake_rates (mmol/gDCW * hr),3hp_secretion_rates (mmol/gDCW * hr)
Line Name,,,
SF ABF93_1-R1,0.002230,0.498801,0.000000
SF ABF93_1-R2,0.002119,0.650572,0.000000
SF ABF93_1-R3,0.002189,0.588836,0.000000
SF ABF93_2-R1,0.002369,0.479287,0.105135
SF ABF93_2-R2,0.001979,0.872079,0.210367
SF ABF93_2-R3,0.002032,0.811947,0.181935
SF ABF93_3-R1,0.002038,0.806031,0.194002
SF ABF93_3-R2,0.002022,0.821504,0.194428
SF ABF93_3-R3,0.001995,0.931884,0.239449


## 

In [22]:
fluxes = {}
for rep in global_prot.columns:
    with model:
        glucose_uptake = normalized_line_rates.loc[rep,'glucose_uptake_rates (mmol/gDCW * hr)' ]
        secrete_3hp = normalized_line_rates.loc[rep,'3hp_secretion_rates (mmol/gDCW * hr)']
        if glucose_uptake:
            model.reactions.BOUNDARY_GLCe.upper_bound = glucose_uptake
        if secrete_3hp:
            model.reactions.EX_3hpp_e.lower_bound =  secrete_3hp
        try:
            print(rep)
            fluxes[rep] = EFlux2(model, global_prot[rep])
        except TypeError:
            print(f"Replicate {rep} with glucose {glucose_uptake} and 3hp {secrete_3hp} is infeasible")
fluxes[rep]

Read LP format model from file /var/folders/z1/nhct2gps3y94tkbwqvvd7y_jw6p1l8/T/tmp56ecgdnu.lp
Reading time = 0.02 seconds
: 1820 rows, 4650 columns, 16544 nonzeros


/Users/zuck016/.pyenv/versions/anaconda3-2020.11/envs/idp_new/lib/python3.6/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


FBA status optimal
FBA solution 1.6503235125841993
EFlux2 status optimal
EFlux2 solution 60.828835949265965
Read LP format model from file /var/folders/z1/nhct2gps3y94tkbwqvvd7y_jw6p1l8/T/tmpzgq464rd.lp
Reading time = 0.02 seconds
: 1820 rows, 4650 columns, 16544 nonzeros
FBA status optimal
FBA solution 1.6628590432314292
EFlux2 status optimal
EFlux2 solution 61.617227212235235
Read LP format model from file /var/folders/z1/nhct2gps3y94tkbwqvvd7y_jw6p1l8/T/tmphh8b20c5.lp
Reading time = 0.02 seconds
: 1820 rows, 4650 columns, 16544 nonzeros
FBA status optimal
FBA solution 1.6656338982217251
EFlux2 status optimal
EFlux2 solution 61.67169460437266
Read LP format model from file /var/folders/z1/nhct2gps3y94tkbwqvvd7y_jw6p1l8/T/tmpne6rlci8.lp
Reading time = 0.02 seconds
: 1820 rows, 4650 columns, 16544 nonzeros
FBA status optimal
FBA solution 1.6672399495681314
EFlux2 status optimal
EFlux2 solution 71.33733673064702
Read LP format model from file /var/folders/z1/nhct2gps3y94tkbwqvvd7y_jw6p1

KeyError: 'ID'

In [26]:
fluxes

{'SF ABF93_1-R1': <Solution 60.829 at 0x7fc907fe49b0>,
 'SF ABF93_1-R2': <Solution 61.617 at 0x7fc904e73b00>,
 'SF ABF93_1-R3': <Solution 61.672 at 0x7fc942bede48>,
 'SF ABF93_10-R1': <Solution 71.337 at 0x7fc926a667f0>,
 'SF ABF93_10-R2': <Solution 72.500 at 0x7fc9345a4278>,
 'SF ABF93_10-R3': <Solution 70.376 at 0x7fc880c64f60>,
 'SF ABF93_11-R1': <Solution 66.241 at 0x7fc92744da90>,
 'SF ABF93_11-R2': <Solution 67.546 at 0x7fc9452a3b38>,
 'SF ABF93_11-R3': <Solution 66.868 at 0x7fc9331f7c50>,
 'SF ABF93_12-R1': <Solution 70.786 at 0x7fc88690f438>,
 'SF ABF93_12-R2': <Solution 73.972 at 0x7fc8600cdd68>,
 'SF ABF93_12-R3': <Solution 70.586 at 0x7fc861dca710>,
 'SF ABF93_13-R1': <Solution 69.020 at 0x7fc8714036a0>,
 'SF ABF93_13-R2': <Solution 66.739 at 0x7fc8877f1f98>,
 'SF ABF93_13-R3': <Solution 69.519 at 0x7fc865a74f28>,
 'SF ABF93_14-R1': <Solution 72.958 at 0x7fc867aa7b70>,
 'SF ABF93_14-R2': <Solution 70.879 at 0x7fc9477746d8>,
 'SF ABF93_14-R3': <Solution 76.131 at 0x7fc947b608

In [ ]:
dir()

In [37]:
dir(fluxes['SF ABF93_9-R3'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'fluxes',
 'get_primal_by_id',
 'objective_value',
 'reduced_costs',
 'shadow_prices',
 'status',
 'to_frame']

In [8]:
efluxes = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['fluxes']) for rep in fluxes]))
reduced_costs = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['reduced_costs']) for rep in fluxes]))
efluxes.to_csv('Eflux2_flux_yield.csv')
reduced_costs.to_csv('Eflux2_yield_reduced_costs.csv')
eflux_rates = efluxes*ref_line_glucose_rate
eflux_rates.to_csv('Learn/Eflux2_flux_rates.csv')

NameError: name 'fluxes' is not defined

In [46]:
eflux_rates

,SF ABF93_1-R1,SF ABF93_1-R2,SF ABF93_1-R3,SF ABF93_10-R1,SF ABF93_10-R2,SF ABF93_10-R3,SF ABF93_11-R1,SF ABF93_11-R2,SF ABF93_11-R3,SF ABF93_12-R1,...,SF ABF93_6-R3,SF ABF93_7-R1,SF ABF93_7-R2,SF ABF93_7-R3,SF ABF93_8-R1,SF ABF93_8-R2,SF ABF93_8-R3,SF ABF93_9-R1,SF ABF93_9-R2,SF ABF93_9-R3
NE1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NE2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
r1,0.700464,0.702898,0.681880,0.539011,0.491422,0.554093,0.622975,0.615275,0.611794,0.557263,...,0.559667,0.574128,0.585345,0.564662,0.608766,0.595037,0.605527,0.586240,0.600469,0.587352
r2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
r3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAPAT,5.167330,5.037285,5.175018,11.849184,12.860283,11.228880,9.574800,9.595030,9.595100,11.692658,...,9.010573,10.737004,10.769589,11.016140,9.158263,8.563418,7.263599,9.151367,9.324460,9.424015
HPDH,0.000000,0.000000,0.000000,8.200368,9.273390,7.726842,5.896979,5.753339,5.833698,8.115986,...,6.383523,7.408641,7.122469,7.701109,5.340573,5.264872,3.915965,5.772879,5.584450,5.650204
ALD6,5.167330,5.037285,5.175018,3.648816,3.586893,3.502038,3.677821,3.841691,3.761402,3.576672,...,2.627051,3.328363,3.647120,3.315031,3.817690,3.298546,3.347634,3.378488,3.740010,3.773811
3HPPt,0.000000,0.000000,0.000000,-8.200368,-9.273390,-7.726842,-5.896979,-5.753339,-5.833698,-8.115986,...,-6.383523,-7.408641,-7.122469,-7.701109,-5.340573,-5.264872,-3.915965,-5.772879,-5.584450,-5.650204


In [40]:
normalized_line_rates.loc[ref_line, 'glucose_uptake_rates (mmol/gDCW * hr)']

1.0